In [118]:
import pandas as pd
import psycopg2
from collections import Counter


In [119]:
df = pd.read_csv("C:/Users/sowmk/Documents/python/case/data.csv")
df.columns = ['title','genre','tags','languages','Movie_Series','hidden_gem_score','country_available','runtime',
'director','writer','actors','view_rating','imdb_score','rotten_tomatoes_score','metacritic_score','awards_received',
'awards_nominated_for','box_office','release_date','netflix_relase_date','production_house','netflix_link','imdb_link',
'summary','imdb_votes','image','poster','tmdb_trailer','trailer_site']
df.head(5)

,title,genre,tags,languages,Movie_Series,hidden_gem_score,country_available,runtime,director,writer,...,netflix_relase_date,production_house,netflix_link,imdb_link,summary,imdb_votes,image,poster,tmdb_trailer,trailer_site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,...,2021-03-04,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,"Brendan Walsh, Daley Nixon",...,2021-03-04,NaN,https://www.netflix.com/watch/81305978,https://www.imdb.com/title/tt8945942,"Trapped in a frozen car during a blizzard, a p...",1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube
3,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81336456,https://www.imdb.com/title/tt6132758,"Upon moving into a new place, a 20-something r...",1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"Lithuania,Poland,France,Iceland,Italy,Spain,Gr...",1-2 hour,Stephen Irwin,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81078393,https://www.imdb.com/title/tt2023611,Inspired by her moms rebellious past and a con...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN


In [120]:
#df.groupby('Movie_Series').count()
df['Movie_Series'].value_counts()

Movie     11697
Series     3783
Name: Movie_Series, dtype: int64

In [121]:
#df[df['title'].str.lower().duplicated()].sort_values(by='title')
movies = df[df['Movie_Series'] == "Movie"].reset_index(level = 0)
movie_id = movies.index +1
movies.insert(0,"movie_id",movie_id)

movies_df = movies[['movie_id','title', 'Movie_Series','runtime','view_rating','imdb_score','rotten_tomatoes_score','metacritic_score','box_office','awards_received','release_date','netflix_relase_date']]
movies_df.head()

,movie_id,title,Movie_Series,runtime,view_rating,imdb_score,rotten_tomatoes_score,metacritic_score,box_office,awards_received,release_date,netflix_relase_date
0,1,HOW TO BUILD A GIRL,Movie,1-2 hour,R,5.8,79.0,69.0,"$70,632",1.0,08 May 2020,2021-03-04
1,2,Centigrade,Movie,1-2 hour,Unrated,4.3,NaN,46.0,"$16,263",NaN,28 Aug 2020,2021-03-04
2,3,Moxie,Movie,1-2 hour,NaN,6.3,NaN,NaN,NaN,NaN,22 Sep 2011,2021-03-04
3,4,The Con-Heartist,Movie,> 2 hrs,NaN,7.4,NaN,NaN,NaN,NaN,03 Dec 2020,2021-03-03
4,5,Only a Mother,Movie,1-2 hour,NaN,6.7,NaN,NaN,NaN,2.0,31 Oct 1949,2021-03-03


In [122]:
movies_df.runtime.drop_duplicates()

runtime_conversion = {"1-2 hour":120,"> 2 hrs":180,"< 30 minutes":30,"30-60 mins":60}
movies_df = movies_df.replace({"runtime":runtime_conversion})
movies_df.head()

,movie_id,title,Movie_Series,runtime,view_rating,imdb_score,rotten_tomatoes_score,metacritic_score,box_office,awards_received,release_date,netflix_relase_date
0,1,HOW TO BUILD A GIRL,Movie,120,R,5.8,79.0,69.0,"$70,632",1.0,08 May 2020,2021-03-04
1,2,Centigrade,Movie,120,Unrated,4.3,NaN,46.0,"$16,263",NaN,28 Aug 2020,2021-03-04
2,3,Moxie,Movie,120,NaN,6.3,NaN,NaN,NaN,NaN,22 Sep 2011,2021-03-04
3,4,The Con-Heartist,Movie,180,NaN,7.4,NaN,NaN,NaN,NaN,03 Dec 2020,2021-03-03
4,5,Only a Mother,Movie,120,NaN,6.7,NaN,NaN,NaN,2.0,31 Oct 1949,2021-03-03


In [123]:
#movies_df['view_rating'].fillna("NA",inplace=True)
movies_df = movies_df.fillna({'view_rating':'NA','production_house':'NA'})
movies_df = movies_df.fillna({'imdb_score':0,'rotten_tomatoes_score':0, 'metacritic_score':0 ,
                               'awards_received':0,'box_office':0})
#movies_df = movies_df.fillna({"release_date":"12 Dec 9999"})
movies_df.head()

,movie_id,title,Movie_Series,runtime,view_rating,imdb_score,rotten_tomatoes_score,metacritic_score,box_office,awards_received,release_date,netflix_relase_date
0,1,HOW TO BUILD A GIRL,Movie,120,R,5.8,79.0,69.0,"$70,632",1.0,08 May 2020,2021-03-04
1,2,Centigrade,Movie,120,Unrated,4.3,0.0,46.0,"$16,263",0.0,28 Aug 2020,2021-03-04
2,3,Moxie,Movie,120,NA,6.3,0.0,0.0,0,0.0,22 Sep 2011,2021-03-04
3,4,The Con-Heartist,Movie,180,NA,7.4,0.0,0.0,0,0.0,03 Dec 2020,2021-03-03
4,5,Only a Mother,Movie,120,NA,6.7,0.0,0.0,0,2.0,31 Oct 1949,2021-03-03


In [124]:
rating = {'R' : 'Restricted','Unrated' : 'NA','NA':'NA','PG-13':'Cautious','PG':'Everyone','TV-Y7':'Everyone','Not Rated':'NA',
'TV-14' : 'Cautious','Approved' : 'Everyone','TV-MA' : 'Restricted','G':'Everyone','Passed':'Everyone',
'TV-Y': 'Everyone','TV-PG' : 'Restricted', 'GP': 'Cautious','TV-G' : 'Everyone','MA-17' : 'Cautious','UNRATED' : 'NA',
'M/PG': 'Cautious','TV-Y7-FV': 'Everyone','X': 'Restricted','NC-17':'Restricted','NOT RATED': 'NA',
'E': 'Cautious','E10+': 'Everyone'}
movies_df = movies_df.replace({"view_rating":rating})
movies_df.head()

,movie_id,title,Movie_Series,runtime,view_rating,imdb_score,rotten_tomatoes_score,metacritic_score,box_office,awards_received,release_date,netflix_relase_date
0,1,HOW TO BUILD A GIRL,Movie,120,Restricted,5.8,79.0,69.0,"$70,632",1.0,08 May 2020,2021-03-04
1,2,Centigrade,Movie,120,NA,4.3,0.0,46.0,"$16,263",0.0,28 Aug 2020,2021-03-04
2,3,Moxie,Movie,120,NA,6.3,0.0,0.0,0,0.0,22 Sep 2011,2021-03-04
3,4,The Con-Heartist,Movie,180,NA,7.4,0.0,0.0,0,0.0,03 Dec 2020,2021-03-03
4,5,Only a Mother,Movie,120,NA,6.7,0.0,0.0,0,2.0,31 Oct 1949,2021-03-03


In [125]:
#movies_df['box_office'] = movies_df['box_office'].str.replace("$","")
#movies_df['box_office'] = movies_df['box_office'].str.replace(",","")
#movies_df['box_office'] = movies_df['box_office'].fillna('0')

movies_df['box_office'] = movies_df['box_office'].str.strip("$")
movies_df['box_office'] = movies_df['box_office'].str.replace(",","")
movies_df['box_office'] = movies_df['box_office'].fillna('0')
movies_df['box_office'].astype('int64')
movies_df.head()

,movie_id,title,Movie_Series,runtime,view_rating,imdb_score,rotten_tomatoes_score,metacritic_score,box_office,awards_received,release_date,netflix_relase_date
0,1,HOW TO BUILD A GIRL,Movie,120,Restricted,5.8,79.0,69.0,70632,1.0,08 May 2020,2021-03-04
1,2,Centigrade,Movie,120,NA,4.3,0.0,46.0,16263,0.0,28 Aug 2020,2021-03-04
2,3,Moxie,Movie,120,NA,6.3,0.0,0.0,0,0.0,22 Sep 2011,2021-03-04
3,4,The Con-Heartist,Movie,180,NA,7.4,0.0,0.0,0,0.0,03 Dec 2020,2021-03-03
4,5,Only a Mother,Movie,120,NA,6.7,0.0,0.0,0,2.0,31 Oct 1949,2021-03-03


In [126]:
movies_df['release_date']  = pd.to_datetime(movies_df['release_date'])
movies_df['release_date']  = movies_df['release_date'].fillna(pd.to_datetime("2099-12-31"))

release_yr = movies_df['release_date'].dt.year
release_qtr = movies_df['release_date'].dt.quarter
movies_df.insert(11,"release_year",release_yr)
movies_df.insert(12,"release_qtr",release_qtr)
movies_df.head()

,movie_id,title,Movie_Series,runtime,view_rating,imdb_score,rotten_tomatoes_score,metacritic_score,box_office,awards_received,release_date,release_year,release_qtr,netflix_relase_date
0,1,HOW TO BUILD A GIRL,Movie,120,Restricted,5.8,79.0,69.0,70632,1.0,2020-05-08,2020,2,2021-03-04
1,2,Centigrade,Movie,120,NA,4.3,0.0,46.0,16263,0.0,2020-08-28,2020,3,2021-03-04
2,3,Moxie,Movie,120,NA,6.3,0.0,0.0,0,0.0,2011-09-22,2011,3,2021-03-04
3,4,The Con-Heartist,Movie,180,NA,7.4,0.0,0.0,0,0.0,2020-12-03,2020,4,2021-03-03
4,5,Only a Mother,Movie,120,NA,6.7,0.0,0.0,0,2.0,1949-10-31,1949,4,2021-03-03


In [127]:
movies_df['netflix_relase_date'] = pd.to_datetime(movies_df['netflix_relase_date'])

nt_release_yr = movies_df['netflix_relase_date'].dt.year
nt_release_qtr = movies_df['netflix_relase_date'].dt.quarter

movies_df.insert(14,"netflix_release_yr",nt_release_yr)
movies_df.insert(15,"netflix_release_qtr",nt_release_qtr)

movies_df.head()

,movie_id,title,Movie_Series,runtime,view_rating,imdb_score,rotten_tomatoes_score,metacritic_score,box_office,awards_received,release_date,release_year,release_qtr,netflix_relase_date,netflix_release_yr,netflix_release_qtr
0,1,HOW TO BUILD A GIRL,Movie,120,Restricted,5.8,79.0,69.0,70632,1.0,2020-05-08,2020,2,2021-03-04,2021,1
1,2,Centigrade,Movie,120,NA,4.3,0.0,46.0,16263,0.0,2020-08-28,2020,3,2021-03-04,2021,1
2,3,Moxie,Movie,120,NA,6.3,0.0,0.0,0,0.0,2011-09-22,2011,3,2021-03-04,2021,1
3,4,The Con-Heartist,Movie,180,NA,7.4,0.0,0.0,0,0.0,2020-12-03,2020,4,2021-03-03,2021,1
4,5,Only a Mother,Movie,120,NA,6.7,0.0,0.0,0,2.0,1949-10-31,1949,4,2021-03-03,2021,1


In [149]:
#g = df['genre'].str.split(",",expand = True).stack().reset_index(drop=True, level = 1).rename('genre')
#genre_df = df[['id','title']].join(g)
#genre_df.head(10)

genre_list = movies['genre'].str.split(",",expand =True).stack().reset_index(drop=True, level =1).rename('genre')
#len(set(genre_list.str.lower().to_list()))
genre_list = genre_list.str.strip().to_list()
genre_list = [x.lower() for x in genre_list]
genre_count = dict(Counter(genre_list))
genre_count_sort = sorted(genre_count.items(), key= lambda x:x[1], reverse=True)
genre_top = { k:v for k,v in dict(genre_count_sort).items() }
genre_top_list =pd.DataFrame(list(genre_top.keys()))
genre_top_list = genre_top_list.replace("-","")

In [150]:
movies = movies.fillna({'genre':'NA', 'tags':'NA','languages':'NA' ,'production_house':'NA'})
#movies.info()

In [151]:
movie_genre = movies['genre'].str.split(",",expand =True).stack().reset_index(drop=True, level =1).rename('genre')
movies_genre_df = movies[['movie_id','title']].join(movie_genre)
movies_genre_df.head(10)



,movie_id,title,genre
0,1,HOW TO BUILD A GIRL,Comedy
1,2,Centigrade,Drama
1,2,Centigrade,Thriller
2,3,Moxie,Animation
2,3,Moxie,Short
2,3,Moxie,Drama
3,4,The Con-Heartist,Comedy
3,4,The Con-Heartist,Romance
4,5,Only a Mother,Drama
5,6,Snowroller,Comedy


In [152]:
g = pd.DataFrame({"id" : [1,2,3], "genre" : ['drama','comedy','comedy,drama']})

glist = g['genre'].to_dict()
glist

val ={}

for k,v in glist.items():
    if 'drama' in v:
        val[k]='Y'
    else:
        val[k] = 'N'
val

flag = pd.DataFrame(val, index=[0])
flag =flag.T

g.join(flag)

,id,genre,0
0,1,drama,Y
1,2,comedy,N
2,3,"comedy,drama",Y


In [153]:
gdf = movies[['movie_id','title','genre']]

gdf_list = gdf['genre'].to_dict()
gdf_list


{0: 'Comedy',
 1: 'Drama, Thriller',
 2: 'Animation, Short, Drama',
 3: 'Comedy, Romance',
 4: 'Drama',
 5: 'Comedy',
 6: 'Comedy, Family, Fantasy',
 7: 'Crime, Drama, Fantasy, Mystery, Thriller',
 8: 'Drama',
 9: 'Comedy, Drama',
 10: 'Short, Drama',
 11: 'Crime, Drama, Thriller',
 12: 'Action, Adventure, Fantasy, Sci-Fi',
 13: 'Adventure, Drama, Fantasy, Mystery',
 14: 'Music',
 15: 'Drama, Thriller',
 16: 'Short',
 17: 'Thriller',
 18: 'Drama',
 19: 'Comedy, Drama',
 20: 'Drama, Romance',
 21: 'Comedy',
 22: 'Action, Family, Sci-Fi, Thriller',
 23: 'Biography, Drama, Thriller',
 24: 'Drama',
 25: 'Crime, Drama',
 26: 'Drama',
 27: 'Drama',
 28: 'Action, Family, Sci-Fi, Thriller',
 29: 'Drama',
 30: 'Drama, Romance',
 31: 'Animation, Drama, Sport',
 32: 'NA',
 33: 'Drama, Romance',
 34: 'Biography, Drama',
 35: 'Comedy',
 36: 'Animation, Adventure, Family, Mystery',
 37: 'Comedy',
 38: 'Drama, War',
 39: 'Drama, History, War',
 40: 'Comedy, Crime, Drama',
 41: 'Adventure, Drama, Sci-

In [154]:
genre_top_list.columns = ['genre']
genre_top_names = genre_top_list['genre'].to_list()
genre_top_names = [x.replace("-","") for x in genre_top_names  ]

genre_top_list1 = genre_top_list['genre'].to_list()
genre_top_list1 = [x+"_flag" for x in genre_top_list1]
genre_top_list1 = [x.replace("-","") for x in genre_top_list1  ]

genre_top_list1
for x in genre_top_list1 : exec(x + "=dict()")



In [155]:
genre_top_dict = { x: 'N' for x in genre_top_names  }
genre_top_dict

{'drama': 'N',
 'comedy': 'N',
 'thriller': 'N',
 'action': 'N',
 'romance': 'N',
 'crime': 'N',
 'adventure': 'N',
 'na': 'N',
 'fantasy': 'N',
 'family': 'N',
 'horror': 'N',
 'scifi': 'N',
 'mystery': 'N',
 'animation': 'N',
 'documentary': 'N',
 'biography': 'N',
 'history': 'N',
 'music': 'N',
 'sport': 'N',
 'war': 'N',
 'short': 'N',
 'musical': 'N',
 'western': 'N',
 'news': 'N',
 'realitytv': 'N',
 'adult': 'N',
 'talkshow': 'N',
 'filmnoir': 'N',
 'gameshow': 'N'}

In [160]:
gflags = {}

for k,v in gdf_list.items():
   #print(v)
   d1 = { x: 'N' for x in genre_top_names  }
   gflags[k] = d1
   for name in v.split(","):
      gflags[k][name.lower().strip().replace("-","")] ='Y'
   
#print(gflags)

gflags_df = pd.DataFrame(gflags)
gflags_df_upd = gflags_df.T
gflags_df_upd

gdf = gdf.join(gflags_df_upd)
gdf.head()

,movie_id,title,genre,drama,comedy,thriller,action,romance,crime,adventure,...,war,short,musical,western,news,realitytv,adult,talkshow,filmnoir,gameshow
0,1,HOW TO BUILD A GIRL,Comedy,N,Y,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
1,2,Centigrade,"Drama, Thriller",Y,N,Y,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
2,3,Moxie,"Animation, Short, Drama",Y,N,N,N,N,N,N,...,N,Y,N,N,N,N,N,N,N,N
3,4,The Con-Heartist,"Comedy, Romance",N,Y,N,N,Y,N,N,...,N,N,N,N,N,N,N,N,N,N
4,5,Only a Mother,Drama,Y,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
